In [12]:
import cell2location as c2l
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import scanpy as sc
import pandas as pd
import numpy as np
import scvi
import anndata as ad
import os
import json
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from scikit_posthocs import posthoc_dunn
from kneed import KneeLocator
from collections import defaultdict
from scipy.stats import kruskal
from statsmodels.stats.multitest import multipletests
import ace_tools_open as tools
from tqdm import tqdm
import glob # For pattern matching filenames
import re   # For regular expressions to extract sample names

In [2]:
# Define input and output file paths
matrix_file_path = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_CosMx_normalized_matrix.txt.gz"
annotation_file_path = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_CosMx_annotation.csv.gz"
output_h5ad_path = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_CosMx_combined.h5ad" # New name for clarity

In [4]:
print(f"Attempting to read expression matrix from: {matrix_file_path}")

try:
    # Read the file. Skip initial 5 comment lines.
    # Read all columns as strings initially to avoid DtypeWarning during initial read.
    # Then convert numerically relevant columns later.
    df_raw = pd.read_csv(
        matrix_file_path,
        sep='\t',
        skiprows=5,
        header=None, # Crucial: Don't let pandas guess a header, we'll parse it manually
        dtype=str,   # Read everything as string to prevent DtypeWarning and mix-type issues
        low_memory=False # For large files, helps pandas infer types better if not specified
    )

    print("Raw DataFrame head (first 5 rows, first 10 columns):\n", df_raw.iloc[:5, :10])
    print(f"Shape of df_raw: {df_raw.shape}")

    # Extract the true header row (gene names and metadata labels)
    true_header_row = df_raw.iloc[0, :].tolist()
    patient_col_label = true_header_row[0] # Should be 'patient'
    cell_id_col_label = true_header_row[1] # Should be 'cell_id'
    fov_col_label = true_header_row[2]     # Should be 'fov'
    gene_names = true_header_row[3:]

    # Extract the actual data rows (starting from the second row, index 1)
    data_rows = df_raw.iloc[1:, :].copy()

    # Step 1.1: Extract Cell Metadata
    cell_metadata_df = data_rows.iloc[:, 0:3].copy()
    cell_metadata_df.columns = [patient_col_label, cell_id_col_label, fov_col_label] # Assign labels

    # Create a unique cell identifier for the AnnData object's obs_names
    # This is crucial for merging later and for AnnData's structure
    # Use original column names for robustness if they change slightly
    cell_metadata_df['unique_cell_id'] = cell_metadata_df[patient_col_label].astype(str) + '_' + \
                                         cell_metadata_df[fov_col_label].astype(str) + '_' + \
                                         cell_metadata_df[cell_id_col_label].astype(str)
    cell_metadata_df = cell_metadata_df.set_index('unique_cell_id')
    # Keep patient, fov, cell_id as columns as well if needed later
    # cell_metadata_df.index.name = None # Remove index name if preferred

    print(f"\nCell metadata shape: {cell_metadata_df.shape}")
    print("Cell metadata head:\n", cell_metadata_df.head())

    # Step 1.2: Extract Expression Matrix
    expression_matrix = data_rows.iloc[:, 3:].copy()
    expression_matrix.columns = gene_names # Assign gene names as columns
    expression_matrix.index = cell_metadata_df.index # Assign unique cell IDs as rows
    
    # Convert expression matrix to numeric (after ensuring gene names are set)
    # Coerce errors to NaN and then fill NaNs with 0
    expression_matrix = expression_matrix.apply(pd.to_numeric, errors='coerce')
    expression_matrix = expression_matrix.fillna(0) # Fill any potential NaNs with 0

    print(f"\nExpression matrix shape: {expression_matrix.shape}")
    print("Expression matrix head (first 5x5):\n", expression_matrix.iloc[:5, :5])

    # Step 2: Create AnnData object
    # AnnData expects Cells x Genes. Our `expression_matrix` is already Cells x Genes.
    adata_st = sc.AnnData(expression_matrix, obs=cell_metadata_df)

    # Ensure gene names are unique (they should be from the list above, but good practice)
    adata_st.var_names_make_unique()

    print("\nAnnData object created:")
    print(adata_st)
    print("AnnData .obs head:\n", adata_st.obs.head())
    print("AnnData .var head:\n", adata_st.var.head()) # Will still be empty DataFrame, but with correct index (gene names)

    # Step 3: Save the AnnData object
    adata_st.write(output_h5ad_path)
    print(f"\nSuccessfully created and saved H5AD: {output_h5ad_path}")

    # Verify by loading
    adata_loaded = sc.read(output_h5ad_path)
    print("\nVerifying loaded H5AD:")
    print(adata_loaded)
    print("Loaded adata.obs head:\n", adata_loaded.obs.head())
    print("Loaded adata.var head:\n", adata_loaded.var.head())


except pd.errors.ParserError as e:
    print(f"Pandas Parser Error: {e}")
    print("The file format might be different than assumed. Please inspect with `gunzip -c {matrix_file_path} | head -n 10`")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Attempting to read expression matrix from: /blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_CosMx_normalized_matrix.txt.gz
Raw DataFrame head (first 5 rows, first 10 columns):
          0        1    2     3     4     5    6                 7  \
0  patient  cell_id  fov  AATK  ABL1  ABL2  ACE              ACE2   
1     CD c        1    1     0     0     0    0  0.63767208105254   
2     CD c        2    1     0     0     0    0                 0   
3     CD c        3    1     0     0     0    0                 0   
4     CD c        4    1     0     0     0    0                 0   

                   8      9  
0              ACKR1  ACKR3  
1                  0      0  
2                  0      0  
3  0.780317497296296      0  
4                  0      0  
Shape of df_raw: (459096, 983)

Cell metadata shape: (459095, 3)
Cell metadata head:
                patient cell_id fov
unique_cell_id                    
CD c_1_1          CD c       1   1
CD c_1_2          CD c       2   

In [9]:
# Define paths
output_h5ad_path = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_CosMx_combined.h5ad"
annotation_file_path = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_CosMx_annotation.csv.gz"

# Load the AnnData object
adata_st = sc.read(output_h5ad_path)
print("AnnData object loaded from combined H5AD:")
print(adata_st)

# --- Inspect adata.obs.index directly ---
print("\n--- Inspecting adata.obs.index (from matrix data) ---")
print("First 10 values of adata_st.obs.index:")
print(adata_st.obs.index[:10].tolist())
print(f"Total number of unique values in adata_st.obs.index: {len(adata_st.obs.index.unique())}")


# --- Exploring the annotation file ---
print(f"\n--- Exploring annotation file: {annotation_file_path} ---")

try:
    df_annotation = pd.read_csv(annotation_file_path, sep=',')

    print("Annotation DataFrame head:\n", df_annotation.head())
    print(f"Annotation DataFrame shape: {df_annotation.shape}")
    print(f"Annotation DataFrame columns: {df_annotation.columns.tolist()}")

    # --- Inspect df_annotation['id'] column directly ---
    if 'id' in df_annotation.columns:
        print("\n--- Inspecting df_annotation['id'] column (from annotation file) ---")
        print("First 10 values of df_annotation['id']:")
        print(df_annotation['id'].head(10).tolist())
        print(f"Total number of unique values in df_annotation['id']: {df_annotation['id'].nunique()}")

        # Merge attempt (will likely still show NaNs if prefixes differ, but confirms merge logic is sound)
        print("\nAttempting merge (expecting NaNs if IDs don't match exactly)...")
        
        # Make sure to work on a copy to avoid SettingWithCopyWarning if we modify df_annotation
        df_annotation_for_merge = df_annotation.copy() 
        df_annotation_for_merge = df_annotation_for_merge.set_index('id')

        # Perform the merge
        adata_st.obs = adata_st.obs.merge(df_annotation_for_merge, left_index=True, right_index=True, how='left', validate='m:1')
        
        print("New adata.obs head after merge attempt:")
        print(adata_st.obs.head())
        print("Number of non-NaN 'subset' values (expected 0 if no matches):", adata_st.obs['subset'].count())

    else:
        print("\nError: The 'id' column was not found in the annotation file. Please check column names.")
        print(f"Columns in annotation file: {df_annotation.columns.tolist()}")

except FileNotFoundError:
    print(f"Error: Annotation file not found at {annotation_file_path}")
except Exception as e:
    print(f"An error occurred while exploring annotation file: {e}")

AnnData object loaded from combined H5AD:
AnnData object with n_obs × n_vars = 459095 × 980
    obs: 'patient', 'cell_id', 'fov'

--- Inspecting adata.obs.index (from matrix data) ---
First 10 values of adata_st.obs.index:
['CD c_1_1', 'CD c_1_2', 'CD c_1_3', 'CD c_1_4', 'CD c_1_5', 'CD c_1_6', 'CD c_1_7', 'CD c_1_8', 'CD c_1_10', 'CD c_1_11']
Total number of unique values in adata_st.obs.index: 459095

--- Exploring annotation file: /blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_CosMx_annotation.csv.gz ---
Annotation DataFrame head:
          id    subset              SingleR2
0  HC_c_2_1    stroma             Pericytes
1  HC_c_3_1    stroma           Endothelium
2  HC_c_4_1    stroma             Pericytes
3  HC_c_5_1       epi  Secretory progenitor
4  HC_c_7_1  myeloids                    M2
Annotation DataFrame shape: (463967, 3)
Annotation DataFrame columns: ['id', 'subset', 'SingleR2']

--- Inspecting df_annotation['id'] column (from annotation file) ---
First 10 values of d

In [18]:
# Define paths
base_10x_dir = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/scRNA/"
annotation_10x_file = os.path.join(base_10x_dir, "GSE214695_cell_annotation.csv.gz")
output_10x_h5ad_path = os.path.join(base_10x_dir, "combined_10x_reference_final.h5ad")

print(f"Starting 10x Genomics data processing from: {base_10x_dir}")

# Find all unique sample prefixes (e.g., GSM6614348_HC-1)
file_pattern = os.path.join(base_10x_dir, "GSM*_matrix.mtx.gz")
matrix_files = glob.glob(file_pattern)
sample_prefixes = sorted([re.match(r"(.*)_matrix\.mtx\.gz", os.path.basename(f)).group(1) for f in matrix_files])

if not sample_prefixes:
    print(f"Error: No 10x Genomics matrix files found matching pattern '{file_pattern}'.")
    exit()

list_of_adatas = []

print(f"Found {len(sample_prefixes)} 10x Genomics samples.")

for prefix in sample_prefixes:
    print(f"\nProcessing sample: {prefix}")
    try:
        matrix_path = os.path.join(base_10x_dir, f"{prefix}_matrix.mtx.gz")
        barcodes_path = os.path.join(base_10x_dir, f"{prefix}_barcodes.tsv.gz")
        features_path = os.path.join(base_10x_dir, f"{prefix}_features.tsv.gz")

        # Read barcodes (cell IDs)
        barcodes_df = pd.read_csv(barcodes_path, header=None, sep='\t')
        cell_ids = barcodes_df[0].values.astype(str)

        # Read features (gene IDs/symbols/types)
        features_df = pd.read_csv(features_path, header=None, sep='\t')
        gene_ids = features_df[0].values.astype(str)
        gene_symbols = features_df[1].values.astype(str) if features_df.shape[1] >= 2 else gene_ids
        feature_types = features_df[2].values.astype(str) if features_df.shape[1] == 3 else None

        print(f"  Features file has {features_df.shape[1]} columns. Using gene_symbols for var_names.")
        print(f"  Expected {len(cell_ids)} cells and {len(gene_symbols)} genes.")

        # Read matrix data using sc.read_mtx
        adata_mtx_raw = sc.read_mtx(matrix_path)

        # Validate and transpose the matrix
        if adata_mtx_raw.shape[0] == len(gene_symbols) and adata_mtx_raw.shape[1] == len(cell_ids):
            print(f"  Matrix shape {adata_mtx_raw.shape} matches (genes x cells). Transposing to (cells x genes).")
            adata_sample = adata_mtx_raw.T
        elif adata_mtx_raw.shape[0] == len(cell_ids) and adata_mtx_raw.shape[1] == len(gene_symbols):
            print(f"  Matrix shape {adata_mtx_raw.shape} matches (cells x genes). No transpose needed.")
            adata_sample = adata_mtx_raw
        else:
            print(f"  ERROR: Matrix shape {adata_mtx_raw.shape} does not match expected cell/gene counts ({len(cell_ids)} cells, {len(gene_symbols)} genes). Skipping this sample.")
            continue

        # Assign cell IDs and gene symbols
        adata_sample.obs_names = cell_ids
        adata_sample.var_names = gene_symbols
        adata_sample.var['gene_ids'] = gene_ids
        if feature_types is not None:
            adata_sample.var['feature_type'] = feature_types

        adata_sample.var_names_make_unique() # Make gene names unique for safety

        # Add sample ID and group to obs metadata
        adata_sample.obs['original_sample_id'] = prefix
        match = re.search(r"_(HC|UC|CD)-\d+$", prefix)
        adata_sample.obs['group'] = match.group(1) if match else 'Unknown'

        # Filter out cells (barcodes) with zero total counts (this resolved the huge N_obs)
        sc.pp.filter_cells(adata_sample, min_counts=1) 
        
        # Add 'n_counts' for consistency with scanpy's default filtering
        adata_sample.obs['n_counts'] = adata_sample.X.sum(axis=1)

        print(f"  Sample {prefix} loaded. Filtered shape: {adata_sample.shape}")
        print("  adata_sample.obs head (filtered):\n", adata_sample.obs.head())
        print("  adata_sample.var head (loaded):\n", adata_sample.var.head())

        list_of_adatas.append(adata_sample)

    except FileNotFoundError as e:
        print(f"  Skipping {prefix}: One or more expected files not found: {e}")
    except Exception as e:
        print(f"  An error occurred processing {prefix}: {e}")

if not list_of_adatas:
    print("No AnnData objects were successfully loaded. Exiting.")
    exit()

print("\nConcatenating all AnnData objects...")
# Concatenate all individual AnnData objects
# REMOVED `label` and `batch_key` from `ad.concat` as we manually handle `original_sample_id`
adata_10x_ref = ad.concat(
    list_of_adatas,
    axis=0,
    join='outer',
    merge='unique'
)

# Manually ensure obs_names are unique across concatenated samples
# Append the original_sample_id to the barcode to make it unique across batches.
# This assumes 'original_sample_id' column was correctly added during the loop.
adata_10x_ref.obs_names = adata_10x_ref.obs['original_sample_id'].astype(str) + '_' + adata_10x_ref.obs_names.astype(str)
adata_10x_ref.obs.index.name = 'cell_barcode_id' # Give the new unique index a meaningful name

# Final check for uniqueness of observation names
adata_10x_ref.obs_names_make_unique()
adata_10x_ref.var_names_make_unique() # Ensure var_names are unique

print("\nCombined 10x Genomics AnnData object created:")
print(adata_10x_ref)
print("adata_10x_ref.obs head:\n", adata_10x_ref.obs.head())
print("adata_10x_ref.var head:\n", adata_10x_ref.var.head())
print(f"Total cells: {adata_10x_ref.n_obs}, Total genes: {adata_10x_ref.n_vars}")

# --- Integrate the GSE214695_cell_annotation.csv.gz ---
print(f"\nIntegrating cell annotation from: {annotation_10x_file}")

try:
    df_anno_10x = pd.read_csv(annotation_10x_file, sep=',')
    print("Annotation file head:\n", df_anno_10x.head())
    print("Annotation file columns:", df_anno_10x.columns.tolist())

    # --- Step 1: Create a MAPPING for sample names ---
    # `original_sample_id` in adata_10x_ref.obs is like 'GSM6614348_HC-1'
    # `sample` in df_anno_10x is like 'HC1', 'CD1', 'UC1'

    # Let's create the mapping based on the structure we see.
    # We can infer the mapping if the number after HC/CD/UC matches.

    sample_id_mapping_anno_to_gsm = {}
    for anno_sample_short in df_anno_10x['sample'].unique():
        # Extract the type (HC, CD, UC) and number (1, 2, 3...)
        match_type_num = re.match(r"(HC|CD|UC)(\d+)", anno_sample_short)
        if match_type_num:
            group_type = match_type_num.group(1) # HC, CD, UC
            group_num = match_type_num.group(2)   # 1, 2, 3 etc.
            
            # Reconstruct the target format like 'HC-1', 'CD-2'
            target_short_id = f"{group_type}-{group_num}"
            
            # Find the full GSM prefix that contains this 'target_short_id'
            matched_gsm_prefix = None
            for gsm_prefix in sample_prefixes: # sample_prefixes is e.g., ['GSM6614348_HC-1', ...]
                if target_short_id in gsm_prefix:
                    matched_gsm_prefix = gsm_prefix
                    break
            
            if matched_gsm_prefix:
                sample_id_mapping_anno_to_gsm[anno_sample_short] = matched_gsm_prefix
            else:
                print(f"Warning: No GSM prefix found for annotation sample '{anno_sample_short}' (derived: {target_short_id}).")
                sample_id_mapping_anno_to_gsm[anno_sample_short] = None # Mark as unmappable
        else:
            print(f"Warning: Annotation sample '{anno_sample_short}' does not match expected (HC1, CD1, UC1) pattern.")
            sample_id_mapping_anno_to_gsm[anno_sample_short] = None

    print("\nDerived sample ID mapping (annotation 'sample' -> full GSM prefix):", sample_id_mapping_anno_to_gsm)

    # Map the 'sample' column in df_anno_10x to the full GSM prefixes
    df_anno_10x['mapped_sample_id'] = df_anno_10x['sample'].map(sample_id_mapping_anno_to_gsm)
    
    # Filter out annotation entries that couldn't be mapped (where mapped_sample_id is None)
    initial_rows_before_map_filter = df_anno_10x.shape[0]
    df_anno_10x.dropna(subset=['mapped_sample_id'], inplace=True)
    if df_anno_10x.shape[0] < initial_rows_before_map_filter:
        print(f"Filtered out {initial_rows_before_map_filter - df_anno_10x.shape[0]} annotation rows due to unmappable sample IDs.")

    if df_anno_10x.empty:
        print("After mapping and filtering, no annotation entries remain. Annotation merge will result in NaNs.")
    else:
        # Create the composite key for merging in the annotation DataFrame
        # Format: GSM_PREFIX_CELL_BARCODE (e.g., GSM6614348_HC-1_AAACCTGAGAAACCAT-1)
        # This should exactly match adata_10x_ref.obs_names (which is 'cell_barcode_id')
        df_anno_10x['merge_key'] = df_anno_10x['mapped_sample_id'].astype(str) + '_' + df_anno_10x['cell_id'].astype(str)
        
        # Handle duplicates in annotation file (as identified before)
        initial_anno_rows = df_anno_10x.shape[0]
        df_anno_10x.drop_duplicates(subset=['merge_key'], inplace=True)
        if df_anno_10x.shape[0] < initial_anno_rows:
            print(f"Dropped {initial_anno_rows - df_anno_10x.shape[0]} duplicate merge_keys in annotation file.")

        df_anno_10x_indexed = df_anno_10x.set_index('merge_key')
        df_anno_10x_indexed.index = df_anno_10x_indexed.index.astype(str) # Ensure string type

        # Perform the merge
        columns_to_merge = [col for col in ['annotation', 'nanostring_reference'] if col in df_anno_10x_indexed.columns]
        
        if columns_to_merge:
            adata_10x_ref.obs = adata_10x_ref.obs.merge(df_anno_10x_indexed[columns_to_merge],
                                                        left_index=True, right_index=True, how='left', validate='m:1')
            print("\nMerge complete using composite key.")
            
        else:
            print("No annotation columns ('annotation', 'nanostring_reference') found to merge.")


    print("\nFinal adata_10x_ref.obs head after annotation merge:")
    print(adata_10x_ref.obs.head())
    print("Final columns in adata_10x_ref.obs:", adata_10x_ref.obs.columns.tolist())
    if 'annotation' in adata_10x_ref.obs.columns:
        print("Number of non-NaN 'annotation' values (should be >0 if matches found):", adata_10x_ref.obs['annotation'].count())
    else:
        print("'annotation' column not found in adata_10x_ref.obs after merge (no matches or columns not present in annotation file).")

    # Save the final combined and annotated 10x reference
    adata_10x_ref.write(output_10x_h5ad_path)
    print(f"\nFinal combined and annotated 10x reference saved to: {output_10x_h5ad_path}")


except FileNotFoundError:
    print(f"Error: 10x annotation file not found at {annotation_10x_file}")
except Exception as e:
    print(f"An unexpected error occurred during 10x annotation integration: {e}")

Starting 10x Genomics data processing from: /blue/pbenos/tan.m/IBDCosMx_scRNAseq/scRNA/
Found 18 10x Genomics samples.

Processing sample: GSM6614348_HC-1
  Features file has 3 columns. Using gene_symbols for var_names.
  Expected 737280 cells and 33538 genes.
  Matrix shape (33538, 737280) matches (genes x cells). Transposing to (cells x genes).
  Sample GSM6614348_HC-1 loaded. Filtered shape: (187434, 33538)
  adata_sample.obs head (filtered):
                    original_sample_id group  n_counts
AAACCTGAGAAACCTA-1    GSM6614348_HC-1    HC       2.0
AAACCTGAGAAGATTC-1    GSM6614348_HC-1    HC       2.0
AAACCTGAGAAGGTTT-1    GSM6614348_HC-1    HC       1.0
AAACCTGAGAATCTCC-1    GSM6614348_HC-1    HC       2.0
AAACCTGAGAATTCCC-1    GSM6614348_HC-1    HC      21.0
  adata_sample.var head (loaded):
                     gene_ids     feature_type
MIR1302-2HG  ENSG00000243485  Gene Expression
FAM138A      ENSG00000237613  Gene Expression
OR4F5        ENSG00000186092  Gene Expression
AL6273

/home/tan.m/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")



Combined 10x Genomics AnnData object created:
AnnData object with n_obs × n_vars = 14626304 × 33538
    obs: 'original_sample_id', 'group', 'n_counts'
    var: 'gene_ids', 'feature_type'
adata_10x_ref.obs head:
                                    original_sample_id group  n_counts
cell_barcode_id                                                      
GSM6614348_HC-1_AAACCTGAGAAACCTA-1    GSM6614348_HC-1    HC       2.0
GSM6614348_HC-1_AAACCTGAGAAGATTC-1    GSM6614348_HC-1    HC       2.0
GSM6614348_HC-1_AAACCTGAGAAGGTTT-1    GSM6614348_HC-1    HC       1.0
GSM6614348_HC-1_AAACCTGAGAATCTCC-1    GSM6614348_HC-1    HC       2.0
GSM6614348_HC-1_AAACCTGAGAATTCCC-1    GSM6614348_HC-1    HC      21.0
adata_10x_ref.var head:
                     gene_ids     feature_type
MIR1302-2HG  ENSG00000243485  Gene Expression
FAM138A      ENSG00000237613  Gene Expression
OR4F5        ENSG00000186092  Gene Expression
AL627309.1   ENSG00000238009  Gene Expression
AL627309.3   ENSG00000239945  Gene Express